In [ ]:
import tensorflow as tf 
import numpy as np 
import pickle, json
import os, time
import re

In [ ]:
with open('config.json') as json_config_file:
    user_config = json.load(json_config_file)
    data_path = user_config.get('data_path', os.getcwd())

# user_config = {}
# user_config['data_path'] = r"<your_path_here>"

# with open('config.json', 'w') as json_config_file:
#     json.dump(user_config, json_config_file)

change user config

In [ ]:
# user_config = {}
# user_config['data_path'] = r"<your_path_here>"

# with open('config.json', 'w') as json_config_file:
#     json.dump(user_config, json_config_file)

In [ ]:
vocab_size=int(1.2e6)
embedding_dim=200

In [ ]:
os.chdir(data_path)
embedding_folder = "d" + str(embedding_dim) +"_word_embedding"
if not os.path.exists(embedding_folder): os.makedirs(embedding_folder)
embedding_file = open(os.path.join("glove.twitter.27B", "glove.twitter.27B." + str(embedding_dim) + "d.txt"),"r",encoding="utf-8")
os.chdir(embedding_folder)

In [ ]:
np_embedding = np.zeros((int(1.2e6), embedding_dim), dtype=np.float16)
word_dict = dict()
word_dict2 = dict()

In [ ]:
start_time = time.time()
line = None
for i in range(int(1.2e6)):
    global line,wordvec,line0
    line0=embedding_file.readline()
    if line0 is None or len(line0)==0 : break
    line = line0[:-1].split(" ")
    tags_pattern = '^<(url|user|smile|lolface|sadface|neutralface|heart|number|repeat|elong|hashtag|allcaps|)>$'
    word=re.sub(tags_pattern, r'ᐸ\1ᐳ', line[0])
    word_dict[line[0]]=i
    word_dict2[word]=i
    wordvec=np.array(line[1:],dtype=np.float16)
    np_embedding[i]=wordvec
print("\nexecution time: {:.9f} seconds".format(time.time() - start_time))

In [ ]:
word_dict_file=open("word_dict.pickle", 'wb')
pickle.dump(word_dict, word_dict_file)
word_dict2_file=open("word_dict2.pickle", 'wb')
pickle.dump(word_dict2, word_dict2_file)

In [ ]:
word_embedding_file = open("word_embeddings_ndarray.pickle", 'wb')
pickle.dump(np_embedding, word_embedding_file)

save vocabulary to a file

In [ ]:
words_file = open("vocabulary.txt", 'w', encoding='utf-8')
for key, _ in sorted(word_dict2.items(), key=lambda i: i[1]):
    words_file.write(key + '\n')
words_file.close()

# Session save

In [ ]:
tf.reset_default_graph()
embedding=tf.Variable(tf.constant(0, dtype=tf.float16, shape=(vocab_size, embedding_dim)), trainable=False, name='embedding')
embedding_ph = tf.placeholder(tf.float16, [vocab_size, embedding_dim])
embedding_init= embedding.assign(embedding_ph)

sess = tf.Session()
_ = sess.run(embedding_init, feed_dict={embedding_ph: np_embedding})

In [ ]:
sess_save_folder = "TF_Variables"
if not os.path.exists(sess_save_folder): os.makedirs(sess_save_folder)
saver=tf.train.Saver({'embedding': embedding})
saver.save(sess, os.path.join(os.getcwd(), sess_save_folder, "Embedding"))